# GUI - Nine Men Morris

In [7]:
%%HTML
<style>
.container { width:100% }
</style>

## Definitionen:
- ring = einer der 3 Quadrate (0-2)
- cell = ein Punkt auf einem Ring (0-7)
- position = (ring, cell)
- board = Array aus 3 Arrays, die angeben, ob, und wenn ja welcher Stein dort sitzt
- remaining = \[noch nicht gesetzte Steine Spieler 1, noch nicht gesetzte Steine Spieler 2\]
- pieces = \[remaining, board\]

In [8]:
%run ./Muehle_Logic.ipynb
%run ./Muehle_Utilities.ipynb
import ipycanvas
from ipycanvas import Canvas, MultiCanvas

## Konstanten

* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]
* PADDING = relativer Abstand des äüßersten Quadrats zum Spielfeldrand
* DISTANCE = relativer Abstand zwischen den Quadraten des Spielfelds

In [9]:
BOARD_SIZE   = 400
DOT_RADIUS   = BOARD_SIZE*0.025
PIECE_RADIUS = BOARD_SIZE*0.04
COLOUR       = ['black', 'white', 'sienna']
COLOUR_HINT  = 'green'
COLOUR_ENEMY = 'red'
PADDING      = 0.05
DISTANCE     = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

## Mögliche Positionen

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

In [10]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

In [11]:
positions = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

wmd_dic = {
    "a1":positions[0][6],
    "a4":positions[0][7],
    "a7":positions[0][0],
    "b2":positions[1][6],
    "b4":positions[1][7],
    "b6":positions[1][0],
    "c3":positions[2][6],
    "c4":positions[2][7],
    "c5":positions[2][0],
    "d1":positions[0][5],
    "d2":positions[1][5],
    "d3":positions[2][5],
    "d5":positions[2][1],
    "d6":positions[1][1],
    "d7":positions[0][1],
    "e3":positions[2][4],
    "e4":positions[2][3],
    "e5":positions[2][2],
    "f2":positions[1][4],
    "f4":positions[1][3],
    "f6":positions[1][2],
    "g1":positions[0][4],
    "g4":positions[0][3],
    "g7":positions[0][2]
}

## Initialisierung Canvas

**Aufbau Leinwand:**

board\[Hintergrund, Linien, Steine]

In [12]:
#board[Hintergrund, Linien, Steine]
board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

# Hintergrund
board[0].fill_style = '#ffffcc'
board[0].fill_rect(0, 0, BOARD_SIZE)

# Strichstärke
board[1].line_width = 5

# Quadrate
board[1].stroke_rect(BOARD_SIZE*col(0), BOARD_SIZE*row(0), BOARD_SIZE*(1-row(0)-col(0))) #ring 0
board[1].stroke_rect(BOARD_SIZE*col(1), BOARD_SIZE*row(1), BOARD_SIZE*(1-row(1)-col(1))) #ring 1
board[1].stroke_rect(BOARD_SIZE*col(2), BOARD_SIZE*row(2), BOARD_SIZE*(1-row(2)-col(2))) #ring 2

# Mittelinien
board[1].begin_path()
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(0)) #oben
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(2))
board[1].move_to(BOARD_SIZE*col(6), BOARD_SIZE*row(3)) #rechts
board[1].line_to(BOARD_SIZE*col(4), BOARD_SIZE*row(3))
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(6)) #unten
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(4))
board[1].move_to(BOARD_SIZE*col(0), BOARD_SIZE*row(3)) #links
board[1].line_to(BOARD_SIZE*col(2), BOARD_SIZE*row(3))
board[1].stroke()

# Punkte (außen, mitte, innen)
for ring in positions:
    for x,y in ring:
        board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)

## Status des Spielfelds
* 0 = kein Stein
* 1 = weißer Stein
* 2 = brauner Stein

In [102]:
class status():
    pieces = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # ring 0
            [0,0,0,0,0,0,0,0], # ring 1
            [0,0,0,0,0,0,0,0]  # ring 2
      ]]
    current_player = 1
    mills = set()

## Spielsteine anzeigen

In [14]:
def draw_piece(ring, cell, player):
    #Hintergrund
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    #Ringe
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS,     0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

In [15]:
def update_board(status):
    with ipycanvas.hold_canvas(board):
        board[2].clear()
        board[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                player = status.pieces[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        board[2].fill_style = 'black'
        board[2].font = '18px serif'
        board[2].fill_text('Spieler '+ str(status.current_player), BOARD_SIZE*0.4, BOARD_SIZE*0.45)
        board[2].fill_text('ist dran.', BOARD_SIZE*0.4, BOARD_SIZE*0.55)
    return board

In [16]:
def on_dot(pos, x, y):
    return (pos[0]*BOARD_SIZE-PIECE_RADIUS/2 < x < pos[0]*BOARD_SIZE+PIECE_RADIUS/2 and pos[1]*BOARD_SIZE-PIECE_RADIUS/2 < y < pos[1]*BOARD_SIZE+PIECE_RADIUS/2)

In [17]:
def handle_mouse_down(x, y):
    global status
    for ring in positions:
        for pos in ring:
            if on_dot(pos, x, y):
                select_stone(status, ring.index(pos), positions.index(ring))
board[2].on_mouse_down(handle_mouse_down)

In [90]:
def select_stone(status, pos, ring):
    phase = get_player_phase(status.pieces, status.current_player)

    if (status.stones_to_remove > 0): #es wurde mindestens eine Mühle erzeugt (vlt. 2)

        #ausgewählten Stein entfernen, wenn es ein feindlicher ist.

        status.stones_to_remove -= 1
        if(status.stones_to_remove == 0):
            status.pieces[0][status.current_player-1] = status.pieces[0][status.current_player-1]-1
            status.current_player = 2 if (status.current_player == 1) else 1
            update_board(status)

    if (phase == 1 and status.pieces[1][ring][pos] == 0 and status.stones_to_remove == 0):
        status.pieces[1][ring][pos] = status.current_player
        update_board(status)

        #Je nach Phase passend auf Mühlen reagieren
        status.stones_to_remove = handle_mills(status)
        if (status.stones_to_remove > 0):
            
            enemy_stones_positions = get_enemy_beatable_stones(status.current_player) #implement #nicht vergessen, keine Steine aus Mühlen nehmen positions = (ring,cell) in einem Array
            show_possible_clickable_positions(enemy_stones_positions, colour = COLOUR_ENEMY)
            #auf klick-Event reagieren mit irgendwas, das vorher in Status gesetzt wurde
                    #in reaktion Stein entfernen und Board updaten
        


            #solange Anzahl zu setzender Steine > 0 & keine Mühle:
            status.pieces[0][status.current_player-1] = status.pieces[0][status.current_player-1]-1

        else: #spieler dar nicht gewechselt werden im Fall einer Mühle
            status.current_player = 2 if (status.current_player == 1) else 1
            update_board(status)

    elif (phase == 2):
        #Spiel in Phase 2
        #Je nach Phase passend auf Mühlen reagieren
        return

    elif (phase == 3):
        #Spiel in Phase 3 
        #Je nach Phase passend auf Mühlen reagieren
        return

    else:
        #default ??
        return

In [86]:
def show_possible_clickable_positions(possible_positions, colour = COLOUR_HINT):
    for position in possible_positions:
        ring, cell = position
        board[2].fill_style = colour
        board[2].global_alpha = TRANSPARENCY_HINT
        board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)

In [103]:
update_board(status)

MultiCanvas(height=400, width=400)

In [99]:
def get_enemy_beatable_stones(current_player):
    enemy_stones_list = []
    if (current_player == 1):
        enemy_player = 2
    else:
        enemy_player = 1

    enemy_mill_stones = set()

    #find all enemy_mill_stones | wenn ein Stein in einer Mühle ist, darf er nicht entfernt werden
    _,positions = status.pieces

    for x in range (0,3):
        for i in [0,2,4,6]:
                if(positions[x][i] == positions[x][i+1] == positions[x][(i+2) % 8] == enemy_player):
                    enemy_mill_stones.add((x,i))
                    enemy_mill_stones.add((x,i+1))
                    enemy_mill_stones.add((x,(i+2) % 8))
    for i in [1,3,5,7]:
        if(positions[0][i] == positions[1][i] == positions[2][i] == enemy_player):
            enemy_mill_stones.add((0,i))
            enemy_mill_stones.add((1,i))
            enemy_mill_stones.add((2,i))

    for ring in range(0,3):
        for cell in range(0,8):
            if positions[ring][cell] == enemy_player:
                if (ring,cell) not in enemy_mill_stones:
                    enemy_stones_list.append((ring,cell)) 
    
    return enemy_stones_list


In [101]:
enemy_stones_positions = get_enemy_beatable_stones(2) 
show_possible_clickable_positions(enemy_stones_positions, colour = COLOUR_ENEMY)